# Notebook 02: Denoising with DADA2

### Objective
The goal of this notebook is to take our raw, paired-end FASTQ files and process them using the DADA2 pipeline. This will correct sequencing errors, merge our paired reads (R1 & R2), and remove chimeras.

The final output will be our two most important files for downstream analysis:
1.  **Feature Table (`table.qza`):** A matrix of Amplicon Sequence Variants (ASVs) by samples (i.e., the counts).
2.  **Representative Sequences (`rep-seqs.qza`):** The unique DNA sequence for each ASV.

### Workflow
1.  **Create a Manifest File:** We need to create a "map" (`manifest.csv`) that tells QIIME 2 where to find the R1 and R2 files for each sample.
2.  **Import Data:** We will use the manifest file to import our 255 samples into a single QIIME 2 artifact (`.qza` file).
3.  **Run DADA2:** We will run `qiime dada2 denoise-paired` using the truncation parameters we found in Notebook 01 (`--p-trunc-len-f 160` and `--p-trunc-len-r 160`).
4.  **Analyze DADA2 Stats:** We will inspect the summary statistics from DADA2 to see how many reads we successfully filtered, denoised, merged, and non-chimeric.

### 1. Create the Paired-End Manifest File

To import our data into QIIME 2, we must first create a "manifest file". This is a CSV file that tells QIIME 2 three things for every sample:
1.  `sample-id`: A unique name for the sample. We will use the `Run` ID (e.g., `SRR7013947`) as our unique sample ID.
2.  `absolute-filepath-r1`: The full, absolute path to its R1 FASTQ file.
3.  `absolute-filepath-r2`: The full, absolute path to its R2 FASTQ file.

We will build this file using our `SraRunTable.csv` and Python.

In [ ]:
import pandas as pd
import os # We'll need this to get the absolute path

# --- 1. Get the Absolute Path ---
# Get the path of the *current* directory (which is 'notebooks/')
# The !pwd command returns a list, so we take the first element [0]
current_dir = !pwd
        
# Go one level up to get the main project directory path
# This will be something like '/home/refm_youssef/16S_microbiome_analysis_crohns_disease'
project_root = os.path.dirname(current_dir[0])

print(f"Project Root Path: {project_root}")

# --- 2. Load the Metadata ---
metadata_file = "../data/SraRunTable.csv"
metadata_df = pd.read_csv(metadata_file)
        
print(f"\nLoaded {len(metadata_df)} records from SraRunTable.")

### 2. Build the Clean and Docker-Ready Manifest File

The standard manifest creation process failed previously due to strict technical requirements from the QIIME 2 version inside our Docker container. We must address three critical points in this step:

1.  **Metadata Format:** QIIME 2 requires all metadata files, including the manifest, to be **Tab-Separated Values (TSV)**, which we ensure by saving the file with `.tsv` extension and setting `sep='\t'`.
2.  **Clean Data:** We filter out the known missing sample (`SRR7014200`) to prevent pipeline failure during file import.
3.  **Docker Path Mapping:** The paths are defined relative to the Docker mount point (`/data`), specifically using `/data/data/raw_fastq/` as the correct path for the files inside the container environment.

This guarantees a successful import into the QIIME 2 artifact.

In [ ]:

# 1. Create a new, empty DataFrame for our manifest
manifest_df = pd.DataFrame()

# 2. Filter out the known missing sample
clean_metadata = metadata_df[metadata_df['Run'] != 'SRR7014200'].copy()
print(f"Filtered out 1 known missing sample. Proceeding with {len(clean_metadata)} samples.")

# 3. Create the 'sample-id' column
manifest_df['sample-id'] = clean_metadata['Run']

# 4. Create the 'forward-absolute-filepath' column (Correct Header + Docker Path)
# Note: We use /data/data/ because we mount $(pwd) to /data
manifest_df['forward-absolute-filepath'] = clean_metadata['Run'].apply(
    lambda run_id: f"/data/data/raw_fastq/{run_id}_1.fastq"
)

# 5. Create the 'reverse-absolute-filepath' column (Correct Header + Docker Path)
manifest_df['reverse-absolute-filepath'] = clean_metadata['Run'].apply(
    lambda run_id: f"/data/data/raw_fastq/{run_id}_2.fastq"
)

# 6. Define the output path (using .tsv extension)
manifest_output_file = "../data/docker_manifest_clean.tsv"

# 7. Save to TSV (Tab Separated)
# We use sep='\t' to make it a TSV file
manifest_df.to_csv(manifest_output_file, index=False, sep='\t')

# 8. Print the first 5 rows of our *new* manifest to verify
print(f"\nClean, Docker-ready TSV manifest saved to: {manifest_output_file}")
manifest_df.head()

### 3. The Import Problem: "Docker Hang"

Our original plan was to import all 255 samples at once. However, during testing in the terminal, we discovered a major technical bottleneck:

* **The Problem:** Running `qiime tools import` on all 255 samples (510 files) at once causes the Docker container to hang (freeze) indefinitely.
* **The Solution:** The only robust workaround is to **split** our 255 samples into small "batches" (e.g., 10 samples per batch) and import them one by one.

This "Split-Apply-Combine" strategy is the correct workflow for this dataset size.

### 4. Step 1: Splitting the Manifest

First, we will split our clean manifest (`docker_manifest_clean.tsv`) into 26 smaller batch files.

In [ ]:
import math

# --- Settings ---
INPUT_MANIFEST = "../data/docker_manifest_clean.tsv"
OUTPUT_DIR = "../data/manifest_batches"
BATCH_SIZE = 10
# ------------------

# Ensure the output directory exists
!mkdir -p {OUTPUT_DIR}

# Read the clean manifest we just created
clean_df = pd.read_csv(INPUT_MANIFEST, sep='\t')
num_samples = len(clean_df)
num_batches = math.ceil(num_samples / BATCH_SIZE)

print(f"Splitting {num_samples} samples into {num_batches} batches of {BATCH_SIZE}...")

for i in range(num_batches):
    # Calculate the start and end index for this batch
    start_idx = i * BATCH_SIZE
    end_idx = (i + 1) * BATCH_SIZE
    
    # Slice the DataFrame
    batch_df = clean_df.iloc[start_idx:end_idx]
    
    # Define the output filename (e.g., batch_01.tsv)
    batch_num = i + 1
    batch_filename = f"{OUTPUT_DIR}/batch_{batch_num:02d}.tsv"
    
    # Save this batch as a TSV
    batch_df.to_csv(batch_filename, index=False, sep='\t')

print(f"\nSuccessfully created {num_batches} manifest files in {OUTPUT_DIR}")

# Verify by listing the files
!ls -lh {OUTPUT_DIR} | head -n 5

### 5. Step 2: DADA2 Denoising (Batch Processing)

Now that our data is split, the correct workflow is:
1.  **Import** a batch.
2.  **Run DADA2** on that imported batch.
3.  Repeat for all batches.
4.  Merge the final results.

Let's start by testing this full pipeline on **Batch 01** only.

**(Warning: This next cell will take some time, as it's running the full DADA2 pipeline on 10 samples.)**

In [ ]:
import time
import os

# --- Settings ---
BATCH_NUM = "01" # We are testing Batch 01 first
TRUNC_LEN_F = 160
TRUNC_LEN_R = 160

# --- Directories (relative to the notebook location) ---
MANIFEST_DIR = "../data/manifest_batches"
IMPORT_DIR = "../results/03_qiime_artifacts"
TABLES_DIR = "../results/04_dada2_tables"
REP_SEQS_DIR = "../results/05_dada2_rep-seqs"
STATS_DIR = "../results/06_dada2_stats"

# --- Paths for this specific batch ---
manifest_file = f"{MANIFEST_DIR}/batch_{BATCH_NUM}.tsv"
imported_qza = f"{IMPORT_DIR}/batch_{BATCH_NUM}.qza"
table_qza = f"{TABLES_DIR}/table-batch_{BATCH_NUM}.qza"
rep_seqs_qza = f"{REP_SEQS_DIR}/rep-seqs-batch_{BATCH_NUM}.qza"
stats_qza = f"{STATS_DIR}/stats-batch_{BATCH_NUM}.qza"

# --- Create Output Directories ---
# Use -p to avoid errors if they already exist
!mkdir -p {IMPORT_DIR} {TABLES_DIR} {REP_SEQS_DIR} {STATS_DIR}

print(f"--- Starting Test Pipeline for Batch {BATCH_NUM} ---")
start_time = time.time()

# --- Step 1/2: Import this batch ---
print(f"Step 1/2: Importing {manifest_file}...")

# We mount the project root (../) to /data
# We set the working directory to /data/notebooks
!docker run --rm -v $(pwd)/..:/data -w /data/notebooks \
  qiime2/core:latest \
  qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path {manifest_file} \
    --output-path {imported_qza} \
    --input-format PairedEndFastqManifestPhred33V2

# --- Verification checkpoint ---
# Check if the import *actually* created the file
# We check the path relative to the notebook (../results/...)
if not os.path.exists(f"../{imported_qza}"):
    print(f"!!! ERROR: Import failed for Batch {BATCH_NUM}. Stopping.")
else:
    print(f"Import successful: {imported_qza}")
    
    # --- Step 2/2: Run DADA2 on this batch ---
    print(f"\nStep 2/2: Running DADA2 on {imported_qza}...")
    !docker run --rm -v $(pwd)/..:/data -w /data/notebooks \
      qiime2/core:latest \
      qiime dada2 denoise-paired \
        --i-demultiplexed-seqs {imported_qza} \
        --p-trunc-len-f {TRUNC_LEN_F} \
        --p-trunc-len-r {TRUNC_LEN_R} \
        --o-table {table_qza} \
        --o-representative-sequences {rep_seqs_qza} \
        --o-denoising-stats {stats_qza} \
        --p-n-threads 0 # Use all available threads

    end_time = time.time()
    print(f"--- Test Pipeline for Batch {BATCH_NUM} finished in {(end_time - start_time):.2f} seconds. ---")

    # --- Final Verification ---
    print("\nVerifying DADA2 outputs:")
    !ls -lh ../{table_qza}
    !ls -lh ../{rep_seqs_qza}

In [ ]:
# ---  Run DADA2 on the Imported Batch 01 ---
# The previous cell successfully imported batch_01.qza.
# Now we run Step 2/2 (DADA2) on that artifact.

print(f"\nStep 2/2: Running DADA2 on {imported_qza}...")
print("(This is the slow step, please wait...)")

# Start timer for DADA2 step
dada2_start_time = time.time()

# --- Run DADA2 Command ---
# (Note: We use the *exact same* variables from the cell above)
!docker run --rm -v $(pwd)/..:/data -w /data/notebooks \
  qiime2/core:latest \
  qiime dada2 denoise-paired \
    --i-demultiplexed-seqs {imported_qza} \
    --p-trunc-len-f {TRUNC_LEN_F} \
    --p-trunc-len-r {TRUNC_LEN_R} \
    --o-table {table_qza} \
    --o-representative-sequences {rep_seqs_qza} \
    --o-denoising-stats {stats_qza} \
    --p-n-threads 0 # Use all available threads

dada2_end_time = time.time()

# --- Final Verification (Corrected) ---
# We check the paths *without* the extra "../"
if os.path.exists(table_qza) and os.path.exists(rep_seqs_qza):
    print(f"\n--- DADA2 Pipeline for Batch {BATCH_NUM} finished in {(dada2_end_time - dada2_start_time):.2f} seconds. ---")
    print("\nVerifying DADA2 outputs:")
    !ls -lh {table_qza}
    !ls -lh {rep_seqs_qza}
else:
    print(f"\n!!! ERROR: DADA2 failed. Output files not found.")

### 6. Run the Full Pipeline on All Batches (02-26)

Our test on Batch 01 was a complete success. We have confirmed that the two-step pipeline (Import, then DADA2) works perfectly.

Now, we will create a `for` loop to apply this exact same pipeline to the remaining 25 batches (from `batch_02.tsv` to `batch_26.tsv`).

**(Warning: This next cell will take a long time to complete!)**
Based on our test (3.6 minutes for 1 batch), we expect this loop to take:
`25 batches * 3.6 minutes/batch ≈ 90 minutes (1.5 hours)`

We will run this cell and let it process all remaining samples.

In [ ]:
# ---  Process Batches 02 through 26 ---
print(f"--- Starting Full Pipeline for Batches 02-26 ---")

# We loop from 2 up to and including 26
for i in range(2, 27):
    # --- Settings for this batch ---
    BATCH_NUM = f"{i:02d}" # (02, 03, 04, ...)
    
    # Define all paths
    manifest_file = f"{MANIFEST_DIR}/batch_{BATCH_NUM}.tsv"
    imported_qza = f"{IMPORT_DIR}/batch_{BATCH_NUM}.qza" 
    table_qza = f"{TABLES_DIR}/table-batch_{BATCH_NUM}.qza"
    rep_seqs_qza = f"{REP_SEQS_DIR}/rep-seqs-batch_{BATCH_NUM}.qza"
    stats_qza = f"{STATS_DIR}/stats-batch_{BATCH_NUM}.qza"
    
    print(f"\n--- Processing Batch {BATCH_NUM} ---")
    batch_start_time = time.time()

    # --- Step 1/2: Import this batch ---
    print(f"Step 1/2: Importing {manifest_file}...")
    !docker run --rm -v $(pwd)/..:/data -w /data/notebooks \
      qiime2/core:latest \
      qiime tools import \
        --type 'SampleData[PairedEndSequencesWithQuality]' \
        --input-path {manifest_file} \
        --output-path {imported_qza} \
        --input-format PairedEndFastqManifestPhred33V2

    # --- Verification checkpoint (Corrected) ---
    # We check the *correct* path: imported_qza (which is '../results/...')
    if not os.path.exists(imported_qza):
        print(f"!!! ERROR: Import failed for Batch {BATCH_NUM}. Stopping loop.")
        # Stop the whole loop if one batch fails
        break 
    else:
        print(f"Import successful: {imported_qza}")
        
        # --- Step 2/2: Run DADA2 on this batch ---
        print(f"\nStep 2/2: Running DADA2 on {imported_qza}...")
        !docker run --rm -v $(pwd)/..:/data -w /data/notebooks \
          qiime2/core:latest \
          qiime dada2 denoise-paired \
            --i-demultiplexed-seqs {imported_qza} \
            --p-trunc-len-f {TRUNC_LEN_F} \
            --p-trunc-len-r {TRUNC_LEN_R} \
            --o-table {table_qza} \
            --o-representative-sequences {rep_seqs_qza} \
            --o-denoising-stats {stats_qza} \
            --p-n-threads 0 # Use all available threads
        
        batch_end_time = time.time()
        
        # Final check for DADA2 outputs
        if os.path.exists(table_qza):
            print(f"--- Batch {BATCH_NUM} finished successfully in {(batch_end_time - batch_start_time):.2f} seconds. ---")
        else:
            print(f"!!! ERROR: DADA2 failed for Batch {BATCH_NUM}. Stopping loop.")
            break

print(f"\n--- Full Batch Processing Complete ---")

### 7. Merge All DADA2 Results

The batch processing (Cell 14) was a complete success. We now have 26 individual Feature Tables (`table-batch_XX.qza`) and 26 individual Representative Sequence files (`rep-seqs-batch_XX.qza`).

The final step in this stage is to merge them into two final, complete artifacts. We will do this in two steps:
1.  Merge all 26 feature tables.
2.  Merge all 26 representative sequence files.

In [ ]:
# --- (Cell 17) Merge all 26 Feature Tables (Corrected) ---

print("--- 1. Merging all 26 Feature Tables ---")
# The previous command failed because --i-tables needs a *list* of files, not a directory.
# We will use a wildcard (*) to pass all 26 files at once.

!docker run --rm -v $(pwd)/..:/data -w /data/notebooks \
  qiime2/core:latest \
  qiime feature-table merge \
    --i-tables ../results/04_dada2_tables/table-batch_*.qza \
    --o-merged-table ../results/table.qza

print("\n--- 2. Verification of Final Merged Table ---")
# Check if the final merged file exists
!ls -lh ../results/table.qza

### 8. Merge Representative Sequences

We have successfully merged our 26 feature tables into `table.qza`. Now, we must do the same for the representative sequences.

This will combine all 26 `rep-seqs-batch_XX.qza` files (from the `results/05_dada2_rep-seqs/` directory) into a single, final `rep-seqs.qza` artifact.

In [ ]:
# ---  Merge all 26 Rep-Seqs (Corrected) ---

print("--- 1. Merging all 26 Rep-Seqs files (Corrected Command) ---")
# The previous command failed due to old parameter names in QIIME 2 v2020.8
# We are using --i-data (instead of --i-sequences)
# We are using --o-merged-data (instead of --o-merged-sequences)

!docker run --rm -v $(pwd)/..:/data -w /data/notebooks \
  qiime2/core:latest \
  qiime feature-table merge-seqs \
    --i-data ../results/05_dada2_rep-seqs/rep-seqs-batch_*.qza \
    --o-merged-data ../results/rep-seqs.qza

print("\n--- 2. Verification of Final Merged Rep-Seqs ---")
# Check if the final merged file exists
!ls -lh ../results/rep-seqs.qza

### 9. Summarize Final Artifacts

We have successfully completed the DADA2 pipeline and merged all results. We now have our two final, critical artifacts:
1.  `table.qza` (The final Feature Table)
2.  `rep-seqs.qza` (The final Representative Sequences)

As a final step in this notebook, we will summarize both of these artifacts to get an overview of our dataset. This will also generate visual `.qzv` files that we can view in `qiime2 view`.

In [ ]:
import pandas as pd

# ---  Create a QIIME 2-compatible Metadata File ---

print("--- 1. Loading original SraRunTable.csv ---")
metadata_df = pd.read_csv("../data/SraRunTable.csv")

# --- 2. Renaming 'Run' column to 'sample-id' ---
# This is the ID column QIIME 2 expects.
# We use 'inplace=True' to modify the DataFrame directly.
metadata_df.rename(columns={'Run': 'sample-id'}, inplace=True)

# --- 3. Saving as a Tab-Separated (TSV) file ---
METADATA_TSV_PATH = "../data/metadata.tsv"
metadata_df.to_csv(METADATA_TSV_PATH, sep='\t', index=False)

print(f"\n--- 4. Successfully created clean metadata file: {METADATA_TSV_PATH} ---")

print("\n--- 5. Verification (showing first 2 lines of the new TSV file) ---")
!head -n 2 {METADATA_TSV_PATH}

In [ ]:
# ---  Summarize the Final Feature Table (Corrected) ---

print("--- 1. Summarizing the merged Feature Table (using metadata.tsv) ---")

!docker run --rm -v $(pwd)/..:/data -w /data/notebooks \
  qiime2/core:latest \
  qiime feature-table summarize \
    --i-table ../results/table.qza \
    --o-visualization ../results/table.qzv \
    --m-sample-metadata-file ../data/metadata.tsv

print("\n--- 2. Verification of Table Summary ---")
!ls -lh ../results/table.qzv

In [ ]:
# ---  Summarize the Final Rep-Seqs (Corrected Plugin Name) ---

print("--- 1. Summarizing the merged Rep-Seqs (Corrected Command) ---")
# The previous command failed due to old plugin names in QIIME 2 v2020.8
# We are using 'qiime feature-table tabulate-seqs'
# (instead of 'qiime feature-data ...')

!docker run --rm -v $(pwd)/..:/data -w /data/notebooks \
  qiime2/core:latest \
  qiime feature-table tabulate-seqs \
    --i-data ../results/rep-seqs.qza \
    --o-visualization ../results/rep-seqs.qzv

print("\n--- 2. Verification of Rep-Seqs Summary ---")
# Check if the final merged file exists
!ls -lh ../results/rep-seqs.qzv

### 11. Conclusion & Next Steps

This notebook was the most computationally intensive part of our pipeline. We successfully addressed the major "Docker Hang" bottleneck by implementing a "Split-Apply-Combine" strategy, which is a robust method for handling large datasets.

We have successfully:
1.  Split our 255 samples into 26 manageable batches.
2.  Run the full DADA2 pipeline (Import + Denoise) on all 26 batches.
3.  Merged the 26 individual results into two final, project-wide artifacts.
4.  Created a clean, QIIME 2-compatible metadata file (`metadata.tsv`).
5.  Summarized and validated all final outputs.

**Final Validated Artifacts:**
* `results/table.qza`: The final ASV Feature Table (Samples x Features).
* `results/rep-seqs.qza`: The final ASV Representative Sequences (Feature ID -> DNA Sequence).
* `results/table.qzv`: A visual summary of the feature table and sample depths.
* `results/rep-seqs.qzv`: A visual summary of the sequence data.

**Next Steps:**
With our clean, validated data ready, we can now move on to **Notebook 03**. In the next stage, we will finally answer the "what" question:
1.  We will analyze the `table.qzv` summary to understand our final read counts and sample quality.
2.  We will perform **Taxonomic Classification** on our `rep-seqs.qza` to assign bacterial names (like *E. coli* or *Lactobacillus*) to our ASVs.